In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv", parse_dates = ["datetime"])
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv", parse_dates = ["datetime"])


In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(train.corr(), annot=True)

In [ ]:
# Feature Engineering
train["year"] = train["datetime"].dt.year
train["hour"] = train["datetime"].dt.hour
train["dayofweek"] = train["datetime"].dt.dayofweek

test["year"] = test["datetime"].dt.year
test["hour"] = test["datetime"].dt.hour
test["dayofweek"] = test["datetime"].dt.dayofweek

y_casual = np.log1p(train.casual)
y_registered = np.log1p(train.registered)

train.drop(["datetime", "windspeed", "casual", "registered", "count"], 1, inplace=True)
test.drop(["datetime", "windspeed", ], 1, inplace=True)

In [ ]:
from lightgbm import LGBMRegressor

hyperparameters = { 'colsample_bytree': 0.725,  'learning_rate': 0.013,
                    'num_leaves': 56, 'reg_alpha': 0.754, 'reg_lambda': 0.071, 
                    'subsample': 0.523, 'n_estimators': 1093, 'boosting_type': 'dart'}

model = LGBMRegressor(**hyperparameters)

In [ ]:
# Instead of predicting count feature, 
# lets predict casual and registered features
model.fit(train, y_casual)
pred_casual = model.predict(test)

model.fit(train, y_registered)
pred_register = model.predict(test)


In [ ]:
submission = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")
submission["count"] = np.expm1(pred_casual) + np.expm1(pred_register)
submission.to_csv("light_gbm_result_rk4.csv", index=False)

print('completed')

In [ ]:
import xgboost as xgb
model = xgb.XGBRegressor(objective ='reg:linear',n_estimators = 10, seed = 123)

In [ ]:

model.fit(train, y_casual)
pred_casual = model.predict(test)

model.fit(train, y_registered)
pred_register = model.predict(test)

submission = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")
submission["count"] = np.expm1(pred_casual) + np.expm1(pred_register)
submission.to_csv("x_gb_result_rk.csv", index=False)

print('xgb completed')